In [4]:
import os

In [5]:
%pwd

'c:\\Users\\shubh\\OneDrive\\Desktop\\kidney-disease-classification\\research'

In [6]:
os.chdir('../')

In [7]:
%pwd

'c:\\Users\\shubh\\OneDrive\\Desktop\\kidney-disease-classification'

In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [11]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [ ]:
# # run this before any model code / before model.fit
# import tensorflow as tf
# tf.config.run_functions_eagerly(True)   # enable eager mode for debugging
# print("eager:", tf.executing_eagerly())
# # after debugging you can disable:
# # tf.config.run_functions_eagerly(False)

eager: True


In [12]:
# ...existing code...
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        # load without loading optimizer state
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path,
            compile=False
        )
        # compile with a fresh optimizer (adjust loss/optimizer to your project)
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(), 
            loss='sparse_categorical_crossentropy', 
            metrics=['accuracy']
        )

    def train_valid_generator(self):
        # ...existing code...
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            class_mode='sparse',
            **dataflow_kwargs,
        )
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs,
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            class_mode='sparse',
            **dataflow_kwargs,
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        # ensure optimizer is fresh (if you changed trainable flags, recreate optimizer and recompile)
        # self.model.compile(optimizer=tf.keras.optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            # workers=4,                   # worker threads for generator
            # use_multiprocessing=True,    # use processes (safer for heavy CPU preprocessing)
            # max_queue_size=10,
            verbose=1
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model,
        )

In [14]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2025-10-20 15:21:31,421: INFO: common: yaml file: config\config.yaml loaded successfully]


[2025-10-20 15:21:31,487: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-20 15:21:31,498: INFO: common: created directory at: artifacts]
[2025-10-20 15:21:31,504: INFO: common: created directory at: artifacts\training]
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.


c:\Users\shubh\anaconda3\envs\kidney\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50


KeyboardInterrupt: 

In [13]:
# verify generator sizes, classes and sample files
from collections import Counter
print("train samples:", training.train_generator.samples)
print("val samples:  ", training.valid_generator.samples)
print("num classes:", training.train_generator.num_classes)
print("train class counts:", Counter(training.train_generator.classes))
print("val class counts:  ", Counter(training.valid_generator.classes))
print("train sample files (first 10):", training.train_generator.filepaths[:10])
print("val sample files   (first 10):", training.valid_generator.filepaths[:10])

NameError: name 'training' is not defined

In [2]:
train_files = set(training.train_generator.filepaths)
val_files = set(training.valid_generator.filepaths)
print("overlap:", len(train_files & val_files))

NameError: name 'training' is not defined

In [1]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

y_true = np.array(training.valid_generator.classes)
y_prob = training.model.predict(training.valid_generator, steps=training.validation_steps+1)
y_pred = np.argmax(y_prob, axis=1)

print(confusion_matrix(y_true[:len(y_pred)], y_pred))
print(classification_report(y_true[:len(y_pred)], y_pred))
if y_prob.shape[1] == 2:
    print("ROC AUC:", roc_auc_score(y_true[:len(y_pred)], y_prob[:len(y_pred),1]))

NameError: name 'training' is not defined